In [465]:
import nltk
import os 
import glob
import re
import  pickle
from nltk.tokenize import sent_tokenize, word_tokenize

In [466]:
def useless_words(delete_words,target):
    for w in delete_words:
        if w in target:
            return False
    return True

def process_documents(text):
    delete_words = ['____','DOCUMENT OF INTERNATIONAL MONETARY FUND','Download Date']
    replace_regex = re.compile(r'\uf0b7|\x0c')
    replace_regex = re.compile(r'\d+?\.\d+|\.\d+|\d+-\d+|\d+/\d+|\d+/|-\d+|\d+|½|¼|¼|⅔|¾/') ## clean up all kinds of numbers
    #test = '123.45 123.25 1234 .245 2015-218  -60 1/asdf 2/asdf  2021/22 ////'
    #replace_regex.findall(test)
    text = replace_regex.sub(' ',text)
    text = re.sub(r'\(.*?\d+.*?\)|\s\s\s\s+\d','',text)  # clean up things like '(box 1)' and  '     1'
    sentances = sent_tokenize(text)
    
    clean_sentances= list()
    for line in sentances:
        if useless_words(delete_words,line):
            tokens = word_tokenize(line)
            if len(tokens)>5:
                wl = [len(t) for t in tokens]
                if max(wl)<30:
                    line = number_regex.sub(' ',line)
                    tokens = word_tokenize(line.lower())
                    clean_sentances.append(tokens)
    
    return clean_sentances

In [467]:
files_path = 'txt_files_finished/'
files = sorted(glob.glob(files_path+'*.txt'))

In [468]:
total_results = list()
for file in files:
    with open(file,'r',encoding='utf-8') as f:
        text = f.read()
    result = process_documents(text)
    total_results.extend(result)
    del result

In [469]:
pickle.dump(total_results,open("sentances.p", "wb"))
#total_results = pickle.load(open("sentances.p", "rb"))

In [478]:
replace_regex = re.compile(r'\d+?\.\d+|\.\d+|\d+-\d+|\d+/\d+|\d+/|-\d+|\d+|½|¼|¼|⅔|¾/') ## clean up all kinds of numbers
#test = '123.45 123.25 1234 .245 2015-218  -60 1/asdf 2/asdf  2021/22 ⅔ ¼ ////'
#replace_regex.findall(test)

['123.45',
 '123.25',
 '1234',
 '.245',
 '2015-218',
 '-60',
 '1/',
 '2/',
 '2021/22',
 '⅔',
 '¼']

### Train Word to vect

In [471]:
import gensim
from gensim.models.word2vec import Word2Vec

In [472]:
### initialize model and build vocabulary 
n_dim = 300
window = 7 
downsampling = 0.001
seed = 1 
num_workers = os.cpu_count()    ## not sure if this is a good idea
min_count = 5 
imf_w2v = Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=n_dim,
    min_count=min_count,
    window= window,
    sample=downsampling
)
## build the vocabulary
imf_w2v.build_vocab(total_results)

In [473]:
## train w2v model 
corpus_count = imf_w2v.corpus_count
iteration = 10
if gensim.__version__[0] =='1':
    imf_w2v.train(total_results)
else:
    imf_w2v.train(total_results,total_examples=corpus_count,epochs = iteration)

In [481]:
imf_w2v.most_similar('policy',topn=50)

[('stances', 0.5617283582687378),
 ('monetary', 0.5204352140426636),
 ('.monetary', 0.49978727102279663),
 ('stimulatory', 0.49678149819374084),
 ('stance', 0.4936649799346924),
 ('blunt', 0.49233806133270264),
 ('nimble', 0.48076796531677246),
 ('quasi-peg', 0.47360220551490784),
 ('recalibrate', 0.47244811058044434),
 ('.fiscal', 0.46385127305984497),
 ('stimuli', 0.45854613184928894),
 ('policies', 0.4516708552837372),
 ('resisting', 0.4499441087245941),
 ('pre-conditions', 0.4491077959537506),
 ('normalizes', 0.4458550214767456),
 ('arrow', 0.44334185123443604),
 ('ift', 0.44234374165534973),
 ('loosen', 0.44200435280799866),
 ('rescinded', 0.4405853748321533),
 ('dilemma', 0.44055449962615967),
 ('‧', 0.43977826833724976),
 ('well-communicated', 0.43827080726623535),
 ('.more', 0.43738624453544617),
 ('looser', 0.43419045209884644),
 ('resolute', 0.43388432264328003),
 ('asynchronous', 0.43379366397857666),
 ('norges', 0.4323478937149048),
 ('pre-emptive', 0.4298253655433655),
 ('

In [475]:
model = imf_w2v.wv
vocabs = model.vocab.keys()